In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os
# path = ""
# path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/'
# os.chdir(path)
# import time
# FOLDERNAME = "OPUS-OpenNMT-bert-0.9-" + str(time.strftime("%Y%m%d-%H%M"))
# !mkdir $FOLDERNAME

# path = path + FOLDERNAME
# os.chdir(path)
# !pwd

import os
path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/OPUS-OpenNMT-bert-0.9-20210412-0158'
os.chdir(path)
!pwd

In [3]:
!nvidia-smi

Wed Apr 28 04:05:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Install libraries**

In [4]:
!pip install OpenNMT-py==1.2.0
!pip install -U scikit-learn

     |████████████████████████████████| 204kB 5.8MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 14.3MB 20.0MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=75589f5125fff4a345c57afd95a0402c4ba86dda7d3934a965c011f22c9fe9fc
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
Successfully built configargparse
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 22.3MB 7.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [5]:
!wget https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/opus_bert.tar.gz
!mkdir data_bin
!tar -xvf 'opus_bert.tar.gz'

--2021-04-28 04:06:14--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/opus_bert.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100661014 (96M) [application/octet-stream]
Saving to: ‘opus_bert.tar.gz’

opus_bert.tar.gz    100%[===================>]  96.00M   112MB/s    in 0.9s    

2021-04-28 04:06:20 (112 MB/s) - ‘opus_bert.tar.gz’ saved [100661014/100661014]

en_train_EM_score_0.95
vi_valid
en_train_EM_0.95
en_train_EM_factor_0.85
en_train_EM_score_0.8
vi_train
en_train_EM_factor_0.8
en_train_EM_0.8
en_valid
en_train_EM_factor_0.95
en_train
en_train_EM_score_0.85
vi_test
en_train_EM_0.85
en_train_EM_score_0.9
en_test
en_train_EM_factor_0.9
en_train_EM_0.9


In [6]:
!mkdir -p output
!onmt_preprocess -train_src 'en_train_EM_0.95' \\
-train_tgt 'vi_train' \\
-valid_src 'en_valid' \\
-valid_tgt 'vi_valid' \\
-save_data 'output/en-vi' 

[2021-04-28 04:06:32,825 INFO] Extracting features...
[2021-04-28 04:06:32,825 INFO]  * number of source features: 0.
[2021-04-28 04:06:32,825 INFO]  * number of target features: 0.
[2021-04-28 04:06:32,825 INFO] Building `Fields` object...
[2021-04-28 04:06:32,825 INFO] Building & saving training data...
[2021-04-28 04:06:34,008 INFO] Building shard 0.
[2021-04-28 04:07:03,448 INFO]  * saving 0th train data shard to output/en-vi.train.0.pt.
[2021-04-28 04:07:20,527 INFO]  * tgt vocab size: 50004.
[2021-04-28 04:07:20,934 INFO]  * src vocab size: 50002.
[2021-04-28 04:07:21,618 INFO] Building & saving validation data...
[2021-04-28 04:07:22,274 INFO] Building shard 0.
[2021-04-28 04:07:24,428 INFO]  * saving 0th valid data shard to output/en-vi.valid.0.pt.


In [7]:
!mkdir -p model
!onmt_train -data 'output/en-vi' \\
-save_model 'model/en-vi' \\
-layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 \\
-encoder_type transformer -decoder_type transformer -position_encoding \\
-train_steps 30000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 \\
-batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 \\
-decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 \\
-param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \\
-report_every 1000 -world_size 1 -gpu_ranks 0

[2021-04-28 04:07:30,699 INFO]  * src vocab size = 50002
[2021-04-28 04:07:30,699 INFO]  * tgt vocab size = 50004
[2021-04-28 04:07:30,699 INFO] Building model...
[2021-04-28 04:07:39,414 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_feat

In [8]:
!ls -al model model/

model:
total 43398608
drwxr-xr-x 2 root root       4096 Apr 28 08:17 .
drwxr-xr-x 1 root root       4096 Apr 28 04:07 ..
-rw-r--r-- 1 root root 1481332543 Apr 28 05:31 en-vi_step_10000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 04:16 en-vi_step_1000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 05:39 en-vi_step_11000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 05:47 en-vi_step_12000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 05:56 en-vi_step_13000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 06:04 en-vi_step_14000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 06:12 en-vi_step_15000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 06:20 en-vi_step_16000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 06:29 en-vi_step_17000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 06:37 en-vi_step_18000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 06:45 en-vi_step_19000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 06:54 en-vi_step_20000.pt
-rw-r--r-- 1 root root 1481332543 Apr 28 04:24 en-vi_step_2000.pt
-rw-r--r--

In [9]:
!onmt_translate -model model/en-vi_step_30000.pt -src en_test -tgt vi_test -output predict.txt

[2021-04-28 08:18:00,016 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [50], which does not match the required output shape [10, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an ou

In [10]:
!tail vi_test

And nobody questions him, because they don't want to hear the answer because it's a lie!
Kubo?
Họ rất vui vẻ, và lúc nào cũng hát với nến.
Nghe này, anh không thể nói chuyện bây giờ được.
Vậy thì con có thể dùng trí tưởng tượng của mình.
Không hề.
Tôi đang nhìn hắn ngay lúc này đây.
Bác không để tâm chứ?
Anh nghĩ cậu ta phản ứng với thuốc?
Bị làm sao mà anh lại đi dự lễ đặt tên em bé của Cuddy chứ?


In [11]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17287, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 17287 (delta 148), reused 135 (delta 77), pack-reused 17041
Receiving objects: 100% (17287/17287), 273.51 MiB | 26.83 MiB/s, done.
Resolving deltas: 100% (12456/12456), done.


In [12]:
!ls -al

total 376852
drwxr-xr-x  1 root root      4096 Apr 28 11:22 .
drwxr-xr-x  1 root root      4096 Apr 28 04:03 ..
drwxr-xr-x  4 root root      4096 Apr 21 13:38 .config
drwxr-xr-x  2 root root      4096 Apr 28 04:06 data_bin
drwx------  6 root root      4096 Apr 28 04:05 drive
-rw-rw-r--  1 1000 1000   3318349 Apr 12 01:48 en_test
-rw-rw-r--  1 1000 1000  26563375 Apr 12 01:48 en_train
-rw-rw-r--  1 1000 1000  34866334 Apr 12 01:07 en_train_EM_0.8
-rw-rw-r--  1 1000 1000  32006284 Apr 12 01:07 en_train_EM_0.85
-rw-rw-r--  1 1000 1000  29696900 Apr 12 01:07 en_train_EM_0.9
-rw-rw-r--  1 1000 1000  27849828 Apr 12 01:07 en_train_EM_0.95
-rw-rw-r--  1 1000 1000  13223858 Apr 12 01:07 en_train_EM_factor_0.8
-rw-rw-r--  1 1000 1000  12138268 Apr 12 01:07 en_train_EM_factor_0.85
-rw-rw-r--  1 1000 1000  11254364 Apr 12 01:07 en_train_EM_factor_0.9
-rw-rw-r--  1 1000 1000  10539008 Apr 12 01:07 en_train_EM_factor_0.95
-rw-rw-r--  1 1000 1000   8152745 Apr 12 01:07 en_train_EM_score_0.8
-rw-rw-r

In [13]:
!perl OpenNMT-py/tools/multi-bleu.perl vi_test < predict.txt

BLEU = 17.49, 41.5/23.1/14.2/9.5 (BP=0.923, ratio=0.926, hyp_len=701996, ref_len=758454)
